In [24]:
chat = ''
with open('chat.txt', mode = 'r', encoding = 'utf-8') as f:
    chat = f.readlines()

In [ ]:
# Generalize to Group Chats
# Complete Date Statistics
# Visualize Data

In [ ]:
sender1 = "Dobi"
sender2 = "Mrudul"
sender1_msgs = []
sender2_msgs = []
sender1_dates = dict()
sender2_dates = dict()

In [ ]:
senders = []
vis = set()          # To mark whether a message traversed is from a new sender or old
messages = dict()    # Store all messages in structured format
dates = dict()       # Store all dates in structures format

In [ ]:
last = None
for msg in chat:
    try:
        # Extract different components of raw data
        d, t = msg.index(','), msg.index('-')
        s = t + msg[t:].index(':')
        date = msg[:d]
        time = msg[d+2:t]
        sender = msg[t+2:s]
        message = msg[s+2:-1]
        
        sender = 
        
        if sender == sender1:
            sender1_msgs.append((date, message))
            if date in sender1_dates:
                sender1_dates[date] += 1
            else:
                sender1_dates[date] = 1
        else:
            sender2_msgs.append((date, message))
            if date in sender2_dates:
                sender2_dates[date] += 1
            else:
                sender2_dates[date] = 1
        last = sender
    except:     # This means there was a newline in chat and is a part of previous message
        if last == sender1:
            sender1_msgs[-1][-1] += msg
        else:
            sender2_msgs[-1][-1] += msg

In [23]:
def sort_dates(dates):
    dates.sort(key = lambda date: list(map(int, date.split('/')))[::-1])
    return dates

dict_keys(['19/11/18', '21/11/18', '22/11/18'])

In [ ]:
def get_date_difference(date1, date2):
    from datetime import date
    d1, m1, y1 = map(int, date1.split('/'))
    d2, m2, y2 = map(int, date2.split('/'))
    diff = date(y2, m2, d2) - date(y1, m1, d1)0
    return abs(diff.days)

In [ ]:
# Date statistics

def biggest_date_difference(dates):
    dates = sort_dates(dates)
    if len(dates) == 1:
        return 0
    max_diff = get_date_difference(dates[0], dates[1])
    
    for i in range(1, len(dates)):
        max_diff = max(max_diff, get_date_difference(date[i-1], date[i]))
    
    return max_diff

def longest_streak(dates):
    pass

def messages_per_day(dates):
    pass

In [ ]:
# Message Counts

def sender_count(sender_msg):
    return len(sender_msg)